In [72]:
import numpy as np
import pandas as pd
import bokeh
import sqlalchemy as sa
import time
from pathlib import Path
import csv
import tempfile

In [73]:
# https://stackoverflow.com/questions/5194057/better-way-to-convert-file-sizes-in-python

import math

def convert_size(size_bytes):
   if size_bytes == 0:
       return "0B"
   size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
   i = int(math.floor(math.log(size_bytes, 1024)))
   p = math.pow(1024, i)
   s = round(size_bytes / p, 2)
   return "%s %s" % (s, size_name[i])

In [74]:
data_type = ["csv", "feather", "pickle", "h5", "parquet", "db"]

# data_type = {'CSV': 'csv', 'FEATHER': "feather", "PICKLE": "pickle", "H5": "h5", 'PARQUET': "parquet", "DB1": "db"}
class DataBase:
    def __init__(self, df=None, ext="csv", fname="df") -> None:
        self.df = df
        self.ext = ext
        self.fname = "df"
        self.location = Path(f"data/output/{self.fname}.{ext}")
        self.write_time = -1
        self.read_time = -1
        self._size = -1
        self.conn = None

    @property
    def size(self):
        return round(self.location.stat().st_size / 1024, 2)

    def write(self):
        if self.ext == "csv":
            start = time.time()
            self.df.to_csv(str(self.location))
            self.write_time = time.time() - start

        elif self.ext == "feather":
            start = time.time()
            self.df.to_feather(str(self.location))
            self.write_time = time.time() - start

        elif self.ext == "pickle":
            start = time.time()
            self.df.to_pickle(str(self.location))
            self.write_time = time.time() - start

        elif self.ext == "h5":
            start = time.time()
            self.df.to_hdf(str(self.location), key="df", mode="w")
            self.write_time = time.time() - start

        elif self.ext == "parquet":
            start = time.time()
            self.df.to_parquet(str(self.location))
            self.write_time = time.time() - start

        elif self.ext == "db":
            # sql
            # conn = sa.create_engine('postgresql://127.0.0.1/pierre', echo=False)
            # https://stackoverflow.com/questions/58896928/how-to-connect-to-sqlite-from-sqlalchemy
            self.conn = sa.create_engine(f"sqlite:///{str(self.location)}", echo=False)
            start = time.time()
            self.df.to_sql("transactions", self.conn, if_exists="replace")
            self.write_time = time.time() - start

    def read(self):
        if self.ext == "csv":
            start = time.time()
            self.df = pd.read_csv(str(self.location))
            self.read_time = time.time() - start

        elif self.ext == "feather":
            start = time.time()
            self.df = pd.read_feather(str(self.location))
            self.read_time = time.time() - start

        elif self.ext == "pickle":
            start = time.time()
            self.df = pd.read_pickle(str(self.location))
            self.read_time = time.time() - start

        elif self.ext == "h5":
            start = time.time()
            self.df = pd.read_hdf(str(self.location), "df")
            self.read_time = time.time() - start

        elif self.ext == "parquet":
            start = time.time()
            self.df = pd.read_parquet(str(self.location))
            self.read_time = time.time() - start

        elif self.ext == "db":
            start = time.time()
            self.df = pd.read_sql("SELECT * from transactions", self.conn)
            # with tempfile.TemporaryFile(mode='w+') as tmpfile:
            # # with open('temp_file.csv', 'w+') as tmpfile:
            #     copy_sql = "COPY (select * from transactions) TO STDOUT WITH CSV HEADER"
            #     copy_sql = "select * from transactions"
            #     # conn = self.conn.raw_connection()
            #     # cur = conn.cursor()
            #     tmp_csv = self.conn.connect().execute(sa.text(copy_sql))
            #     print(tmp_csv.keys())
            #     # print(tmp_csv)
            #     outcsv = csv.writer(tmpfile)
            #     outcsv.writerow(tmp_csv.keys())
            #     outcsv.writerows(tmp_csv)
            #     # cur.copy_export(copy_sql, tmpfile)
            #     # tmpfile.flush()
            #     # tmpfile.seek(0)
            #     # tmpfile.write("temp data")
            #     tmpfile.flush()
            #     tmpfile.seek(0)
            #     # print(tmpfile.readlines())
            #     self.df = pd.read_csv(tmpfile)
            #     # cur.close()
            #     # conn.close()
            self.read_time = time.time() - start
            # if self.conn:
            #     self.conn.close()

In [75]:
# df = pd.read_csv(r'data/input/example_transaction.csv',)
df  = pd.read_csv(r'data/input/airline_dataset.csv')
# df  = pd.read_csv(r'data/input/credit_card_transactions.csv')

dbs = {}
for dtype in data_type:
    tmp_db = DataBase(df, ext=dtype)

    tmp_db.write()

    tmp_db.read()

    dbs[dtype] = tmp_db

In [76]:
print('write times: ')
for dtype in data_type:
    print(f'  Time to write {dtype}: {dbs[dtype].write_time:.5f} seconds')

fastest_write = min(dbs, key=lambda x: dbs[x].write_time)
slowest_write = max(dbs, key=lambda x: dbs[x].write_time)
print(f'The fastest file format for writing is {fastest_write}')
print(f'The slowest file format for writing is {slowest_write}')
print('')


print('read times: ')
for dtype in data_type:
    print(f'  Time to read {dtype}: {dbs[dtype].read_time:.5f} seconds')
    # print(dbs[dtype].df.head)
fastest_read = min(dbs, key=lambda x: dbs[x].read_time)
slowest_read = max(dbs, key=lambda x: dbs[x].read_time)
print(f'The fastest file format for reading is {fastest_read}')
print(f'The slowest file format for reading is {slowest_read}')
print('')

print('size: ')
for dtype in data_type:
    print(f'  {dtype} size: {dbs[dtype].size:.2f} KB')
smallest = min(dbs, key=lambda x: dbs[x].size)
largest = max(dbs, key=lambda x: dbs[x].size)
print(f'The smallest file format is {smallest}')
print(f'The largest file format is {largest}')
print('')


write times: 
  Time to write csv: 0.95506 seconds
  Time to write feather: 0.13116 seconds
  Time to write pickle: 0.15820 seconds
  Time to write h5: 0.33273 seconds
  Time to write parquet: 0.35640 seconds
  Time to write db: 3.27293 seconds
The fastest file format for writing is feather
The slowest file format for writing is db

read times: 
  Time to read csv: 0.52417 seconds
  Time to read feather: 0.16711 seconds
  Time to read pickle: 0.13914 seconds
  Time to read h5: 0.32019 seconds
  Time to read parquet: 0.15522 seconds
  Time to read db: 1.48319 seconds
The fastest file format for reading is pickle
The slowest file format for reading is db

size: 
  csv size: 12969.16 KB
  feather size: 11615.57 KB
  pickle size: 9784.51 KB
  h5 size: 10501.19 KB
  parquet size: 4270.36 KB
  db size: 14840.00 KB
The smallest file format is parquet
The largest file format is db



fastest_format_reading = min(reading_times, key=reading_times.get)
slowest_format_reading = max(reading_times, key=reading_times.get)
print(f'The fastest file format for reading is {fastest_format_reading}')
print(f'The slowest file format for reading is {slowest_format_reading}')